In [6]:
# 1）测试 layers.input_from_feature_columns  
# ref link: https://zhuanlan.zhihu.com/p/393327229

import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.layers.python.layers.feature_column import _EmbeddingColumn 
from tensorflow.contrib.layers.python.layers import initializers

from collections import OrderedDict
import numpy as np
import json

# dense feature ====================================================================== #
# ops
discount_rate_column = layers.real_valued_column(
		column_name=str('discount_rate'),
		dimension=1,
		default_value=0.0
	)

cvr_column = layers.real_valued_column(
		column_name=str('cvr'),
		dimension=1,
		default_value=0.0
		)	

rating_column = layers.real_valued_column(
		column_name=str('rating'),
		dimension=1,
		default_value=0.0
		)
# concat ops
real_value_feature_column = [discount_rate_column, rating_column]

# sparse feature ====================================================================== #
# ops
embedding_hash_column = layers.sparse_column_with_hash_bucket(
                column_name=str('refer_type'),
                hash_bucket_size=1000
                )
embedding_column = _EmbeddingColumn(embedding_hash_column, 
				dimension=10, 
				combiner='mean', 
				max_norm=None,
				initializer=initializers.xavier_initializer()
                )
# concat ops
embedding_feature_column = [embedding_column]

# ======== hyper ops ============================================================== #
hyper_feature_column = [discount_rate_column, rating_column, embedding_column]

# ======== 真实输入数据 + ops (input_from_feature_columns) ========================== #
# for dense 
t1 = tf.Variable(tf.zeros([4,1]), dtype=tf.float32)
t2 = tf.Variable(tf.ones([4,1]), dtype=tf.float32)
t3 = tf.Variable(2*tf.ones([4,1]), dtype=tf.float32)
fg_real_value_dict = {'discount_rate':t1, 'cvr':t2, 'rating':t3}
#
input_layer_real_value = layers.input_from_feature_columns(fg_real_value_dict, real_value_feature_column)

# for sparse
refer_type_column = tf.SparseTensor(indices=tf.constant([[0,0],[1,0],[2,0],[3,0]],dtype=tf.int64), 
		values=tf.constant(['R','B','R','G'],dtype=tf.string), dense_shape=tf.constant([4,2], dtype=tf.int64))
fg_embedding_dict = {'refer_type':refer_type_column}
#
input_layer_sparse_embedding = layers.input_from_feature_columns(fg_embedding_dict, embedding_feature_column)

# for hyper
hyper_fg_features_dict = {'discount_rate':t1, 'cvr':t2, 'rating':t3, 'refer_type':refer_type_column}
#
input_layer_features_emb = layers.input_from_feature_columns(hyper_fg_features_dict, hyper_feature_column)

# ======= run ops + 测试效果 ======= #
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    #
    prt_v1 = sess.run(input_layer_real_value)
    prt_v2 = sess.run(input_layer_sparse_embedding)
    prt_v3 = sess.run(input_layer_features_emb)
    #
    print("v1 shape:", input_layer_real_value.get_shape().as_list())
    print("v2 shape:", input_layer_sparse_embedding.get_shape().as_list())
    print("v3 shape:", input_layer_features_emb.get_shape().as_list()) # 12 = (1+1) + (10), 2个dense + 1个sparse_emb
    print("v1:", prt_v1)
    print("v2:", prt_v2)
    print("v3:", prt_v3)

v1 shape: [4, 2]
v2 shape: [4, 10]
v3 shape: [4, 12]
v1: [[0. 2.]
 [0. 2.]
 [0. 2.]
 [0. 2.]]
v2: [[ 0.03725886 -0.04974178  0.05874137 -0.03631894 -0.07612269 -0.06513395
   0.0550221  -0.03387212 -0.05819836 -0.00593839]
 [ 0.04789885  0.02594811  0.06986267 -0.05396718 -0.05740303 -0.00773607
   0.05224996  0.06423826  0.07566387  0.0620318 ]
 [ 0.03725886 -0.04974178  0.05874137 -0.03631894 -0.07612269 -0.06513395
   0.0550221  -0.03387212 -0.05819836 -0.00593839]
 [ 0.02420689  0.05710719  0.07322019  0.06378414 -0.06661127 -0.00470754
  -0.03304504 -0.04493327 -0.02838157 -0.06198477]]
v3: [[ 2.7611315e-02  5.6142360e-03 -1.9727867e-02 -3.2069616e-02
   2.3613803e-02 -4.1444238e-02  7.5172655e-02 -8.9154765e-03
   7.3011778e-02  5.3057693e-02  0.0000000e+00  2.0000000e+00]
 [-6.7851938e-02 -5.4922722e-02  7.2742201e-02 -2.7516920e-02
   2.6176251e-02  2.2736728e-02  7.2202675e-02 -7.1903072e-02
   1.6700231e-02 -1.8034633e-02  0.0000000e+00  2.0000000e+00]
 [ 2.7611315e-02  5.614

In [2]:
# 2）测试tf.tile
import tensorflow as tf
a = tf.constant([7,19])
a1 = tf.tile(a, multiples=[3]) # 第一个维度复制3遍, (1,2)=>(1,6)
b = tf.constant([[4,5], [3,5]])
b1 = tf.tile(b, multiples=[2,3]) # 第一个维度复制2遍，第二个维度复制3遍, (2,2)=>(4,6)

with tf.Session() as sess:
    a_out = sess.run(a)
    a1_out = sess.run(a1)
    b_out = sess.run(b)
    b1_out = sess.run(b1)
    print("a: ", a_out)
    print("a1: ", a1_out)
    print("b: ", b_out)
    print("b1: ", b1_out)

a:  [ 7 19]
a1:  [ 7 19  7 19  7 19]
b:  [[4 5]
 [3 5]]
b1:  [[4 5 4 5 4 5]
 [3 5 3 5 3 5]
 [4 5 4 5 4 5]
 [3 5 3 5 3 5]]


In [5]:
# 3) 测试 tf.expand_dims
import tensorflow as tf

t = tf.constant([7,19]) #shape: (2,)
e1 = tf.expand_dims(t,0).shape.as_list() # (1,2) # 第0维扩展
e2 = tf.expand_dims(t,1).shape.as_list() # (2,1) # 第1维扩展
e3 = tf.expand_dims(t,-1).shape.as_list() # (2,1)

t2 = tf.Variable(tf.ones([2,3,5], dtype=tf.float32)) #shape: (2,3,5)
e4 = tf.expand_dims(t2,0).shape.as_list() # (1,2,3,5) 
e5 = tf.expand_dims(t2,2).shape.as_list() # (2,3,1,5) 
e6 = tf.expand_dims(t2,3).shape.as_list() # (2,3,5,1)

print("e1~e6: \n{} \n{} \n{} \n{} \n{} \n{}".format(e1, e2, e3, e4, e5, e6))

e1~e6: 
[1, 2] 
[2, 1] 
[2, 1] 
[1, 2, 3, 5] 
[2, 3, 1, 5] 
[2, 3, 5, 1]


In [6]:
# 测试 tf.squeeze
import tensorflow as tf 

t = tf.Variable(tf.ones([1,2,1,3,1,1]), dtype=tf.float32)
s0 = t.shape.as_list() # (1,2,1,3,1,1)
s1 = tf.squeeze(t).shape.as_list() # 去除所有维度的冗余dim => (2,3)
s2 = tf.squeeze(t, [2,4]).shape.as_list() # 去除2和4维度的1 => (1,2,3,1)

print("s0~s2: \n{} \n{} \n{}".format(s0, s1, s2))

s0~s2: 
[1, 2, 1, 3, 1, 1] 
[2, 3] 
[1, 2, 3, 1]
